In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 4.8 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
from tensorflow import keras

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving all-data.csv to all-data.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['all-data.csv']), encoding='latin_1', header=None)

In [ ]:
df.head()

,0,1
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [ ]:
df = df.rename(columns={0: 'sentiment', 1: 'tweet'})
df.head()

,sentiment,tweet
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [ ]:
df.shape

(4846, 2)

In [ ]:
df.isna().sum()

sentiment    0
tweet        0
dtype: int64

In [ ]:
df.sentiment.value_counts()

neutral     2879
positive    1363
negative     604
Name: sentiment, dtype: int64

In [ ]:
df['result'] = df['sentiment'].apply(lambda x: 0 if x == 'neutral' else (1 if x == 'positive' else 2))
df.head()

,sentiment,tweet,result
0,neutral,"According to Gran , the company has no plans t...",0
1,neutral,Technopolis plans to develop in stages an area...,0
2,negative,The international electronic industry company ...,2
3,positive,With the new production plant the company woul...,1
4,positive,According to the company 's updated strategy f...,1


In [ ]:
answer = pd.get_dummies(df.sentiment)
answer.head()

,negative,neutral,positive
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,0,0,1


In [ ]:
y = answer.values
y

array([[0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['tweet'], y, stratify=y)

In [ ]:
bert_prep = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_model = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
def get_sentence_embeding(sent):
  prep_text = bert_prep(sent)
  return bert_model(prep_text)['pooled_output']

In [ ]:
get_sentence_embeding(['i like apple'])

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-8.71409118e-01, -2.98809022e-01,  9.86819640e-02,
         6.81400299e-01,  4.48228009e-02, -1.87569648e-01,
         8.89232218e-01,  2.26127639e-01, -7.58772865e-02,
        -9.99883473e-01,  1.12313533e-03,  4.08791006e-01,
         9.79040682e-01, -6.67450503e-02,  9.33522046e-01,
        -6.14859998e-01, -1.24849379e-01, -5.93419492e-01,
         3.23982596e-01, -7.23952651e-01,  6.27399325e-01,
         9.67094600e-01,  4.52360749e-01,  2.92361081e-01,
         4.09323394e-01,  6.12668395e-01, -6.06403768e-01,
         9.21502054e-01,  9.39320385e-01,  6.63750648e-01,
        -6.82503164e-01,  1.49046138e-01, -9.82890546e-01,
        -2.16752842e-01,  7.69138038e-02, -9.84391928e-01,
         2.65518337e-01, -7.65734911e-01, -4.11513969e-02,
         2.64315084e-02, -8.95289540e-01,  2.95499563e-01,
         9.98467267e-01, -2.45845243e-01, -2.57827714e-02,
        -3.05424899e-01, -9.99978244e-01,  2.24250585e-01,
      

In [ ]:
from keras.layers import Dense, Activation

from keras.models import Sequential

In [ ]:
#bert layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text = bert_prep(text_input)

outputs = bert_model(preprocessed_text)


#nn layer
l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])

l = tf.keras.layers.Dense(3, activation='softmax', name='output')(l)


#model = Sequential()
#model.add(Dense(768))
#model.add(Activation('relu'))
#model.add(Dense(3))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#model.metrics_names

model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [ ]:
METRICS = [
           tf.keras.metrics.BinaryAccuracy(name='accuracy')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
57/57 [==============================] - 1561s 27s/step - loss: 0.9979 - accuracy: 0.5487
Epoch 2/10
57/57 [==============================] - 1558s 27s/step - loss: 0.8988 - accuracy: 0.5861
Epoch 3/10
57/57 [==============================] - 1544s 27s/step - loss: 0.8758 - accuracy: 0.6021
Epoch 4/10
57/57 [==============================] - 1564s 27s/step - loss: 0.8475 - accuracy: 0.6090
Epoch 5/10
57/57 [==============================] - 1549s 27s/step - loss: 0.8438 - accuracy: 0.6114
Epoch 6/10
57/57 [==============================] - 1543s 27s/step - loss: 0.8088 - accuracy: 0.6241
Epoch 7/10
57/57 [==============================] - 1565s 27s/step - loss: 0.8017 - accuracy: 0.6354
Epoch 8/10
57/57 [==============================] - 1565s 27s/step - loss: 0.7933 - accuracy: 0.6359
Epoch 9/10
57/57 [==============================] - 1569s 28s/step - loss: 0.7927 - accuracy: 0.6357
Epoch 10/10
57/57 [==============================] - 1575s 28s/step - loss: 0.7851 - accura

In [ ]:
model.evaluate(X_test, y_test)

38/38 [==============================] - 533s 14s/step - loss: 0.7420 - accuracy: 0.6823


[0.7420291900634766, 0.6823432445526123]